<a href="https://colab.research.google.com/github/kalihidroxit/NLPApp/blob/master/Chatbot_Vietnamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function

import sys
import os
import pandas as pd
import numpy as np
import re
import nltk
import zipfile

from keras.layers import Input, Embedding, LSTM, TimeDistributed, Dense, Bidirectional
from keras.models import Model, load_model

import os

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
base_path = "/content/gdrive/My Drive/chatbot/"

In [4]:
print(os.listdir(base_path))

['vocabulary.txt', 'InsuranceQAquestionanslabelraw.encoded', 'InsuranceQAlabel2answerraw.encoded', 'cornell-moviedialog-corpus.zip', 'cornell-moviedialog-corpus', 'data.txt', 'model_attention.h5']


In [0]:
INPUT_LENGTH = 20
OUTPUT_LENGTH = 20

# **Load the data**

In [0]:
lines = open(base_path + 'data.txt', encoding='utf-8', errors='ignore').read().split('\n')
# conv_lines = open(base_path + 'movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

# **Create a list of all of the conversations' lines' ids.**

In [0]:
convs = []
for line in lines[:-1]:
    _line = line.split('\t')
    convs.append(_line)

#**id and conversation sample**

In [9]:
for k in convs[0]:
    print (k)#, id2line[k])

Xin chào.
Chào bạn.
Bạn khỏe chứ?
Tôi khỏe. Bạn cũng khỏe phải không?
Tôi khỏe.Bạn biết nói Tiếng Anh chứ?
Một ít. Bạn là người Mỹ à?
Vâng!
Bạn đến từ đâu?
Tôi đến từ California
Hân hạnh được gặp bạn.
Tôi cũng vậy


#Sort the sentences into questions (inputs) and answers (targets)

In [10]:
questions = []
answers = []
for conv in convs:
    for i in range(len(conv)-1):
        questions.append(conv[i])
        answers.append(conv[i+1])
        
# Compare lengths of questions and answers
print(len(questions))
print(len(answers))

3858
3858


In [11]:
print(questions[0])
print(answers[0])

Xin chào.
Chào bạn.


In [12]:
print(questions[3])
print(answers[3])

Tôi khỏe. Bạn cũng khỏe phải không?
Tôi khỏe.Bạn biết nói Tiếng Anh chứ?


# Find the length of sentences (not using nltk due to processing speed)

In [13]:
lengths = []
# lengths.append([len(nltk.word_tokenize(sent)) for sent in clean_questions]) #nltk approach
for question in questions:
    lengths.append(len(question.split()))
for answer in answers:
    lengths.append(len(answer.split()))
# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))

13.0
16.0
18.0
22.0


# Randomly check

In [15]:
r = np.random.randint(1,len(questions))

for i in range(r, r+3):
    print(questions[i])
    print(answers[i])
    print()

Chà, ngày mai tôi phải phẫu thuật trên ngón tay.
Ngón tay bạn bị sao à?

Ngón tay bạn bị sao à?
Tôi bị gãy khi chơi bóng rổ

Tôi bị gãy khi chơi bóng rổ
Thật là tồi tệ



# Preprocessing for word based model

In [16]:
import nltk
nltk.download('punkt')

num_samples = 100000  # Number of samples to train on.
questions = questions[:num_samples]
answers = answers[:num_samples]
#tokenizing the qns and answers
questions_tok = [nltk.word_tokenize(sent) for sent in questions]
answers_tok = [nltk.word_tokenize(sent) for sent in answers]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#train-validation split

In [17]:
data_size = len(questions_tok)

# We will use the first 0-80th %-tile (80%) of data for the training
training_input  = questions_tok[:round(data_size*(80/100))]
training_input  = [tr_input[::-1] for tr_input in training_input] #reverseing input seq for better performance
training_output = answers_tok[:round(data_size*(80/100))]

# We will use the remaining for validation
validation_input = questions_tok[round(data_size*(80/100)):]
validation_input  = [val_input[::-1] for val_input in validation_input] #reverseing input seq for better performance
validation_output = answers_tok[round(data_size*(80/100)):]

training_size = len(training_input)
validation_size = len(validation_input)
print('training size', training_size)
print('validation size', validation_size)

training size 3086
validation size 772


# Word en/decoding dictionaries
Create a dictionary for the frequency of the vocabulary


In [0]:
vocab = {}
for question in questions_tok:
    for word in question:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

for answer in answers_tok:
    for word in answer:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1      

In [63]:
len(vocab)

2315

# Remove rare words from the vocabulary.


In [19]:
threshold = 5
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 2315
Size of vocab we will use: 1216


#we will create dictionaries to provide a unique integer for each word.

In [20]:
WORD_CODE_START = 1
WORD_CODE_PADDING = 0


word_num  = 2 #number 1 is left for WORD_CODE_START for model decoder later
encoding = {}
decoding = {1: '<STA>'}
for word, count in vocab.items():
    if count >= threshold: #get vocabularies that appear above threshold count
        encoding[word] = word_num 
        decoding[word_num ] = word
        word_num += 1

print("No. of vocab used:", word_num)

No. of vocab used: 1218


In [21]:
#include unknown token for words not in dictionary
decoding[word_num] = '<UNK>'
encoding['<UNK>'] = word_num
dict_size = word_num+1
dict_size

1219

# Vectorizing dataset

In [0]:
def transform(encoding, data, vector_size=20):
    """
    :param encoding: encoding dict built by build_word_encoding()
    :param data: list of strings
    :param vector_size: size of each encoded vector
    """
    transformed_data = np.zeros(shape=(len(data), vector_size))
    for i in range(len(data)):
        for j in range(min(len(data[i]), vector_size)):
            try:
                transformed_data[i][j] = encoding[data[i][j]]
            except:
                transformed_data[i][j] = encoding['<UNK>']
    return transformed_data

In [23]:
#encoding training set
encoded_training_input = transform(
    encoding, training_input, vector_size=INPUT_LENGTH)
encoded_training_output = transform(
    encoding, training_output, vector_size=OUTPUT_LENGTH)

print('encoded_training_input', encoded_training_input.shape)
print('encoded_training_output', encoded_training_output.shape)

encoded_training_input (3086, 20)
encoded_training_output (3086, 20)


In [24]:
#encoding validation set
encoded_validation_input = transform(
    encoding, validation_input, vector_size=INPUT_LENGTH)
encoded_validation_output = transform(
    encoding, validation_output, vector_size=OUTPUT_LENGTH)

print('encoded_validation_input', encoded_validation_input.shape)
print('encoded_validation_output', encoded_validation_output.shape)

encoded_validation_input (772, 20)
encoded_validation_output (772, 20)


# 2 Model Building
2.1 Sequence-to-Sequence in Keras

In [25]:
import tensorflow as tf
tf.keras.backend.clear_session()

INPUT_LENGTH = 20
OUTPUT_LENGTH = 20

encoder_input = Input(shape=(INPUT_LENGTH,))
decoder_input = Input(shape=(OUTPUT_LENGTH,))

from keras.layers import SimpleRNN

encoder = Embedding(dict_size, 128, input_length=INPUT_LENGTH, mask_zero=True)(encoder_input)
encoder = LSTM(512, return_sequences=True, unroll=True)(encoder)
encoder_last = encoder[:,-1,:]

print('encoder', encoder)
print('encoder_last', encoder_last)

decoder = Embedding(dict_size, 128, input_length=OUTPUT_LENGTH, mask_zero=True)(decoder_input)
decoder = LSTM(512, return_sequences=True, unroll=True)(decoder, initial_state=[encoder_last, encoder_last])

print('decoder', decoder)

# For the plain Sequence-to-Sequence, we produced the output from directly from decoder
# output = TimeDistributed(Dense(output_dict_size, activation="softmax"))(decoder)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
encoder Tensor("lstm_1/transpose_2:0", shape=(?, 20, 512), dtype=float32)
encoder_last Tensor("strided_slice:0", shape=(?, 512), dtype=float32)
decoder Tensor("lstm_2/transpose_2:0", shape=(?, 20, 512), dtype=float32)


2.2 Attention Mechanism

In [26]:
from keras.layers import Activation, dot, concatenate

# Equation (7) with 'dot' score from Section 3.1 in the paper.
# Note that we reuse Softmax-activation layer instead of writing tensor calculation
attention = dot([decoder, encoder], axes=[2, 2])
attention = Activation('softmax', name='attention')(attention)
print('attention', attention)

context = dot([attention, encoder], axes=[2,1])
print('context', context)

decoder_combined_context = concatenate([context, decoder])
print('decoder_combined_context', decoder_combined_context)

# Has another weight + tanh layer as described in equation (5) of the paper
output = TimeDistributed(Dense(512, activation="tanh"))(decoder_combined_context)
output = TimeDistributed(Dense(dict_size, activation="softmax"))(output)
print('output', output)


attention Tensor("attention/truediv:0", shape=(?, 20, 20), dtype=float32)
context Tensor("dot_2/MatMul:0", shape=(?, 20, 512), dtype=float32)
decoder_combined_context Tensor("concatenate_1/concat:0", shape=(?, 20, 1024), dtype=float32)
output Tensor("time_distributed_2/Reshape_1:0", shape=(?, 20, 1219), dtype=float32)


In [27]:
model = Model(inputs=[encoder_input, decoder_input], outputs=[output])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')#, metrics = ['accuracy'])
model.summary()



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 20, 128)      156032      input_2[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 128)      156032      input_1[0][0]                    
__________________________________________________________________________________________

In [0]:
training_encoder_input = encoded_training_input
training_decoder_input = np.zeros_like(encoded_training_output)
training_decoder_input[:, 1:] = encoded_training_output[:,:-1]
training_decoder_input[:, 0] = WORD_CODE_START
training_decoder_output = encoded_training_output.reshape(training_size, INPUT_LENGTH, 1)
#training_decoder_output = np.eye(dict_size)[encoded_training_output.astype('int')]

validation_encoder_input = encoded_validation_input
validation_decoder_input = np.zeros_like(encoded_validation_output)
validation_decoder_input[:, 1:] = encoded_validation_output[:,:-1]
validation_decoder_input[:, 0] = WORD_CODE_START
validation_decoder_output = encoded_validation_output.reshape(validation_size, OUTPUT_LENGTH, 1)
#validation_decoder_output = np.eye(dict_size)[encoded_validation_output.astype('int')]

In [29]:
model.fit(x=[training_encoder_input, training_decoder_input], y=[training_decoder_output],    
          validation_data=([validation_encoder_input, validation_decoder_input], [validation_decoder_output]),
          #validation_split=0.05,
          batch_size=128, epochs=100)

model.save(base_path + 'model_attention.h5')




Train on 3086 samples, validate on 772 samples
Epoch 1/100





3086/3086 [==============================] - 18s 6ms/step - loss: 6.0351 - val_loss: 5.4523
Epoch 2/100
3086/3086 [==============================] - 1s 406us/step - loss: 5.2812 - val_loss: 5.2643
Epoch 3/100
3086/3086 [==============================] - 1s 403us/step - loss: 5.0078 - val_loss: 5.0920
Epoch 4/100
3086/3086 [==============================] - 1s 412us/step - loss: 4.7722 - val_loss: 4.9269
Epoch 5/100
3086/3086 [==============================] - 1s 414us/step - loss: 4.5702 - val_loss: 4.7690
Epoch 6/100
3086/3086 [==============================] - 1s 396us/step - loss: 4.3745 - val_loss: 4.6341
Epoch 7/100
3086/3086 [==============================] - 1s 413us/step - loss: 4.1923 - val_loss: 4.5242
Epoch 8/100
3086/3086 [==============================] - 1s 395us/step - loss: 4.0280 - val_loss: 4.4534
Epoch 9/100
3086/3086 [==============================] - 1s 397us/step - loss: 3.8839 - val_loss: 4.4181
E

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:883: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'strided_slice:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'strided_slice:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


# 3. Model testing¶

In [0]:
def prediction(raw_input):
    clean_input = raw_input
    input_tok = [nltk.word_tokenize(clean_input)]
    input_tok = [input_tok[0][::-1]]  #reverseing input seq
    encoder_input = transform(encoding, input_tok, 20)
    decoder_input = np.zeros(shape=(len(encoder_input), OUTPUT_LENGTH))
    decoder_input[:,0] = WORD_CODE_START
    for i in range(1, OUTPUT_LENGTH):
        output = model.predict([encoder_input, decoder_input]).argmax(axis=2)
        decoder_input[:,i] = output[:,i]
    return output

def decode(decoding, vector):
    """
    :param decoding: decoding dict built by word encoding
    :param vector: an encoded vector
    """
    text = ''
    for i in vector:
        if i == 0:
            break
        text += ' '
        text += decoding[i]
    return text

In [60]:
    questions = 'bạn biết nói tiếng anh không?'
    output = prediction(questions)
    print ('Q:', questions)
    print ('A:', decode(decoding, output[0]))

Q: bạn biết nói tiếng anh không?
A:  Vâng , tôi được một <UNK> đang hay . Bạn nghĩ thế nào ?
